In [118]:
import numpy as np
import pandas as pd

df = pd.read_csv('iris.csv')

In [119]:
df = df[['Id','SepalLengthCm','SepalWidthCm','Species']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   Species        150 non-null    object 
dtypes: float64(2), int64(1), object(1)
memory usage: 4.8+ KB


In [120]:
#Bài 1a: Tính trung bình cộng chiều dài, chiều rộng đài hoa của từng loại (3 loại hoa)

mean_df = df.groupby('Species')[['SepalLengthCm','SepalWidthCm']].mean()
mean_df

,SepalLengthCm,SepalWidthCm
Species,,
Iris-setosa,5.006,3.418
Iris-versicolor,5.936,2.770
Iris-virginica,6.588,2.974


In [121]:
#Bài 2a: Xác định chiều dài, chiều rộng đài hoa lớn nhất của từng loại (3 loại hoa)

sepal_width_df = df.groupby('Species')['SepalWidthCm'].agg(
    max_sepal_width = lambda x: np.max(x),
    min_sepal_width = lambda x: np.min(x)
)

sepal_length_df = df.groupby('Species')['SepalLengthCm'].agg(
    max_sepal_length = lambda x: np.max(x),
    min_sepal_length = lambda x: np.min(x)
)

sepal_width_df

,max_sepal_width,min_sepal_width
Species,,
Iris-setosa,4.4,2.3
Iris-versicolor,3.4,2.0
Iris-virginica,3.8,2.2


In [122]:
sepal_length_df

,max_sepal_length,min_sepal_length
Species,,
Iris-setosa,5.8,4.3
Iris-versicolor,7.0,4.9
Iris-virginica,7.9,4.9


Bài 3: Nhập input đầu vào là 2 thông số, chiều dài & chiều rộng của 1 bông hoa bất kỳ (theo đơn vị mm). Xác định xem bông hoa đấy thuộc loại hoa nào (Iris setosa, Iris virginica hay Iris versicolor). Loại của bông hoa được xác định là class của
- a) Điểm có tọa độ gần nó nhất
- b) Trung bình cộng nhỏ nhất đến tất cả các điểm của từng loại

a) Xác định dựa theo điểm có toạ độ gần nó nhất dựa theo công thức Euclidean distance:

In [123]:
df['SepalLengthWidth'] = df.apply(lambda x: [x['SepalLengthCm'],x['SepalWidthCm']],axis=1)
df.head(5)

,Id,SepalLengthCm,SepalWidthCm,Species,SepalLengthWidth
0,1,5.1,3.5,Iris-setosa,"[5.1, 3.5]"
1,2,4.9,3.0,Iris-setosa,"[4.9, 3.0]"
2,3,4.7,3.2,Iris-setosa,"[4.7, 3.2]"
3,4,4.6,3.1,Iris-setosa,"[4.6, 3.1]"
4,5,5.0,3.6,Iris-setosa,"[5.0, 3.6]"


In [124]:
from scipy.spatial import distance

def identify_class(length,width,df):
    flower = [length,width]
    df_temp = df.copy()
    df_temp['Distance'] = df_temp.apply(lambda x: distance.euclidean(flower,x['SepalLengthWidth']),axis=1)
    df_temp.sort_values(by='Distance',ascending=True,inplace=True)
    
    print(df_temp.head(5))
    print(f"This flower's specie is: {df_temp["Species"].values[0]}")

In [125]:
identify_class(5,2,df)

      Id  SepalLengthCm  SepalWidthCm          Species SepalLengthWidth  \
60    61            5.0           2.0  Iris-versicolor       [5.0, 2.0]   
93    94            5.0           2.3  Iris-versicolor       [5.0, 2.3]   
57    58            4.9           2.4  Iris-versicolor       [4.9, 2.4]   
98    99            5.1           2.5  Iris-versicolor       [5.1, 2.5]   
106  107            4.9           2.5   Iris-virginica       [4.9, 2.5]   

     Distance  
60   0.000000  
93   0.300000  
57   0.412311  
98   0.509902  
106  0.509902  
This flower's specie is: Iris-versicolor


b) Xác định theo trung bình cộng nhỏ nhất đến tất cả các điểm của từng loại

In [126]:
def identify_class_2(length,width,df):
    flower = [length,width]
    df_temp = df.copy()
    df_temp['Distance'] = df_temp.apply(lambda x: distance.euclidean(flower,x['SepalLengthWidth']),axis=1)
    
    mean_species = df_temp.groupby('Species')['Distance'].agg(
        mean = lambda x: np.mean(x)
    )
    mean_species.reset_index(inplace=True)
    mean_species.sort_values(by='mean',ascending=True,inplace=True)
    print("This flower's species is:", mean_species['Species'].values[0])

In [127]:
identify_class_2(5,2,df)

This flower's species is: Iris-versicolor


In [128]:
#Bài 1b: In ra chiều dài đài hoa lớn nhất của 10 bông hoa mỗi loại
df['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [129]:
for i in df['Species'].unique():
    print(df[df['Species']==i]['SepalLengthCm'].nlargest(10))

14    5.8
15    5.7
18    5.7
33    5.5
36    5.5
5     5.4
10    5.4
16    5.4
20    5.4
31    5.4
Name: SepalLengthCm, dtype: float64
50    7.0
52    6.9
76    6.8
65    6.7
77    6.7
86    6.7
58    6.6
75    6.6
54    6.5
51    6.4
Name: SepalLengthCm, dtype: float64
131    7.9
117    7.7
118    7.7
122    7.7
135    7.7
105    7.6
130    7.4
107    7.3
109    7.2
125    7.2
Name: SepalLengthCm, dtype: float64


In [134]:
#Bài 2b: Thống kê trong 50 bông hoa có tổng chiều dài + chiều rộng đài hoa lớn nhất, bao nhiêu bông thuộc về từng loại

df['TotalWidthLength'] = df.apply(lambda x: x['SepalLengthCm'] + x['SepalWidthCm'],axis=1)
df.sort_values(by='TotalWidthLength',ascending=False,inplace=True)
b2 = df[:50].groupby('Species')['TotalWidthLength'].count().sort_values(ascending=False)
b2

Species
Iris-virginica     30
Iris-versicolor    13
Iris-setosa         7
Name: TotalWidthLength, dtype: int64

Bài 3b: Nhập input đầu vào là 2 thông số, chiều dài & chiều rộng của 1 bông hoa bất kỳ (theo đơn vị mm). Xác định xem bông hoa đấy thuộc loại hoa nào (Iris setosa, Iris virginica hay Iris versicolor). Loại của bông hoa được xác định là class của
- a) Điểm có tọa độ gần nó nhất. Nếu có 2 điểm thuộc 2 loại hoa khác nhau cùng nằm gần nhất với input, thì xác định sang điểm kế tiếp
- b) 7 điểm gần nó nhất (Vote đa số). Nếu không xác định được class chiếm đa số, in ra bông hoa này ko thuộc class nào
- c) Lấy tọa độ của input là tâm của 1 đường tròn, có bán kính 2 mm. Vote đa số dựa trên tất cả các điểm nằm trên và trong đường tròn. Nếu có 2 class có số điểm bằng nhau và đồng cao nhất, in ra thông báo bông hoa này ko thuộc class nào.

In [179]:
#3b - a: Sử dụng và chỉnh code từ task 3a

def identify_class(length,width,df):
    
    identified_class = ''
    
    flower = [length,width]
    df_temp = df.copy()
    df_temp['Distance'] = df_temp.apply(lambda x: distance.euclidean(flower,x['SepalLengthWidth']),axis=1)
    df_temp.sort_values(by='Distance',ascending=True,inplace=True)
        
    if (df_temp['Distance'].values[0] == df_temp['Distance'].values[1]) and (df_temp['Species'].values[0] != df_temp['Species'].values[1]):
        identified_class = df_temp['Species'].values[2]
    else:
        identified_class = df_temp['Species'].values[0]
        
    
    print(df_temp.head(7))
    print(f"This flower's specie is: {identified_class}")

In [180]:
identify_class(5.1,3.5,df)

    Id  SepalLengthCm  SepalWidthCm      Species SepalLengthWidth  \
0    1            5.1           3.5  Iris-setosa       [5.1, 3.5]   
17  18            5.1           3.5  Iris-setosa       [5.1, 3.5]   
43  44            5.0           3.5  Iris-setosa       [5.0, 3.5]   
40  41            5.0           3.5  Iris-setosa       [5.0, 3.5]   
39  40            5.1           3.4  Iris-setosa       [5.1, 3.4]   
27  28            5.2           3.5  Iris-setosa       [5.2, 3.5]   
4    5            5.0           3.6  Iris-setosa       [5.0, 3.6]   

    TotalWidthLength  Distance  
0                8.6  0.000000  
17               8.6  0.000000  
43               8.5  0.100000  
40               8.5  0.100000  
39               8.5  0.100000  
27               8.7  0.100000  
4                8.6  0.141421  
This flower's specie is: Iris-setosa


In [181]:
#3b-b: 7 điểm gần nó nhất (Vote đa số). Nếu không xác định được class chiếm đa số, in ra bông hoa này ko thuộc class nào

def identify_class_2(length,width,df):
    identified_class = ''
    flower = [length,width]
    df_temp = df.copy()
    df_temp['Distance'] = df_temp.apply(lambda x: distance.euclidean(flower,x['SepalLengthWidth']),axis=1)
    
    count_species = df_temp[:7].groupby('Species')['Distance'].agg(
        count = lambda x: np.count_nonzero(x)
    ).reset_index()
    
    max_count = [x for x in count_species['count'].values if x == max(count_species['count'].values)]
    
    if len(max_count) > 1:
        print("This flower does not belong to any class")
    else:
        identified_class = count_species[count_species['count']==count_species['count'].max()]
    
    print(identified_class)

In [182]:
identify_class_2(5.1,3.5,df)

          Species  count
0  Iris-virginica      7


In [193]:
'''
3b-c: Lấy tọa độ của input là tâm của 1 đường tròn, có bán kính 2 mm:
   - Vote đa số dựa trên tất cả các điểm nằm trên và trong đường tròn. 
   - Nếu có 2 class có số điểm bằng nhau và đồng cao nhất, in ra thông báo bông hoa này ko thuộc class nào.
'''

# Nếu khoảng cách từ tâm đường tròn đến các điểm khác =< 2 thì các điểm nằm trên và trong đường tròn

def identify_class_3(length,width,df):
    identified_class = ''
    input = [length,width]
    
    df_temp = df.copy()
    df_temp['Distance'] = df_temp.apply(lambda x: distance.euclidean(input,x['SepalLengthWidth']),axis=1)
    df_temp = df_temp[df_temp['Distance']<=2]
    
    count_species = df_temp.groupby('Species')['Distance'].agg(
        count = lambda x: np.count_nonzero(x)
    ).reset_index()
    
    print(count_species)
    
    max_count = [x for x in count_species['count'].values if x == max(count_species['count'].values)]
    
    if len(max_count) > 1:
        print("This flower does not belong to any class")
    else:
        identified_class = count_species[count_species['count']==count_species['count'].max()]
        identified_class = identified_class[identified_class['count'] == max(count_species['count'].values)]['Species'].values[0]
        print(f"This flower belong to group {identified_class}")   

In [194]:
identify_class_3(5.1,3.5,df)

           Species  count
0      Iris-setosa     48
1  Iris-versicolor     50
2   Iris-virginica     38
This flower belong to group Iris-versicolor
